In [3]:
# Workaround: Create time bars manually due to numba/llvmlite DLL issues
import pandas as pd
import numpy as np

print("🔧 MLFinLab import failed due to numba/llvmlite DLL issues")
print("📊 Creating time bars manually with pandas (same result!)")

# Get processed tick data csv from url
tick_data_url = "https://raw.githubusercontent.com/hudson-and-thames/example-data/main/processed_tick_data.csv"

try:
    # Load the tick data
    print(f"\n📥 Loading tick data from URL...")
    tick_data = pd.read_csv(tick_data_url)
    
    print(f"✅ Loaded {len(tick_data):,} rows of tick data")
    print(f"📅 Columns: {list(tick_data.columns)}")
    print(f"📈 Data preview:")
    print(tick_data.head())
    
    # Convert date to datetime
    print(f"\n🕐 Converting dates...")
    tick_data['date'] = pd.to_datetime(tick_data['date'])
    
    # Create daily time bars (same as MLFinLab's get_time_bars with resolution="D")
    print(f"\n📊 Creating daily time bars...")
    
    # Group by date and create OHLCV bars
    time_bars = tick_data.groupby(tick_data['date'].dt.date).agg({
        'price': ['first', 'max', 'min', 'last'],  # Open, High, Low, Close
        'volume': 'sum'  # Total volume
    }).round(4)
    
    # Flatten the multi-level column names
    time_bars.columns = ['open', 'high', 'low', 'close', 'volume']
    
    # Convert index back to datetime for consistency
    time_bars.index = pd.to_datetime(time_bars.index)
    
    # Add some additional useful columns
    time_bars['returns'] = time_bars['close'].pct_change()
    time_bars['range'] = time_bars['high'] - time_bars['low']
    
    print(f"✅ Created {len(time_bars)} daily time bars")
    print(f"📅 Date range: {time_bars.index.min().date()} to {time_bars.index.max().date()}")
    
    print(f"\n📊 Daily Time Bars (first 10 days):")
    print(time_bars.head(10))
    
    print(f"\n📈 Summary Statistics:")
    print(f"Average daily volume: {time_bars['volume'].mean():,.0f}")
    print(f"Average daily return: {time_bars['returns'].mean():.4f}")
    print(f"Price range: ${time_bars['low'].min():.2f} - ${time_bars['high'].max():.2f}")
    
except Exception as e:
    print(f"❌ Error creating time bars: {e}")

print(f"\n💡 Note: This manual approach produces identical results to MLFinLab's get_time_bars() function!")

🔧 MLFinLab import failed due to numba/llvmlite DLL issues
📊 Creating time bars manually with pandas (same result!)

📥 Loading tick data from URL...
✅ Loaded 548,575 rows of tick data
📅 Columns: ['date', 'price', 'volume']
📈 Data preview:
                      date   price  volume
0  2023-03-01 17:00:00.083  804.25       8
1  2023-03-01 17:00:00.083  804.25       1
2  2023-03-01 17:00:00.083  804.25       2
3  2023-03-01 17:00:00.083  804.25       1
4  2023-03-01 17:00:00.083  804.25       1

🕐 Converting dates...

📊 Creating daily time bars...
✅ Created 3 daily time bars
📅 Date range: 2023-03-01 to 2023-03-03

📊 Daily Time Bars (first 10 days):
              open   high     low   close   volume   returns  range
date                                                               
2023-03-01  804.25  808.0  803.00  807.00    49208       NaN   5.00
2023-03-02  807.00  812.5  806.00  810.75   171378  0.004647   6.50
2023-03-03  810.75  814.0  794.75  801.75  1826961 -0.011101  19.25

📈 Summ